In [1]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0. Importamos librerías y ajustamos configuraciones.

In [3]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Importar otras librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------

import csv


# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', 1000) # Ajusta el ancho de la consola para evitar truncamientos


### 1. Creamos la conexión con SQL

In [4]:
# Tratamos de crear la conexión con SQL. Usamos try/except para manejar los posibles errores.

config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'raise_on_warnings': True
}

try:
  cnx = mysql.connector.connect(**config)

except mysql.connector.Error as err:

  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Hay un problema con tu nombre de usuario o contraseña")
  
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("La base de datos no existe")
  
  else:
    print(err)

### 2. Creamos nuestra base de datos: BD_cinemextract

In [5]:
# crear un cursor para poder ejecutar nuestras consultas 

mycursor = cnx.cursor()

# crear nuestra base de datos (schema)

try:
    mycursor.execute("CREATE DATABASE BD_cinemextract")
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)



1007 (HY000): Can't create database 'bd_cinemextract'; database exists
Error Code: 1007
SQLSTATE HY000
Message Can't create database 'bd_cinemextract'; database exists


### 3. Nos conectamos con la base de datos BD_cinemextract.

In [5]:

def establecer_conexion(database):

    # Esta función conecta con la base de datos que se le indica por parámetro y devuelve el conector creado

    config = {
    'user': 'root',
    'password': 'AlumnaAdalab',
    'host': '127.0.0.1',
    'database' : f'{database}',
    'raise_on_warnings': True
    }

    try:
        cnx = mysql.connector.connect(**config)
        print(f"Conexión establecida con {database}")

    except mysql.connector.Error as err:

        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Hay un problema con tu nombre de usuario o contraseña")
        
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("La base de datos no existe")
        
        else:
            print(err)
    
    return cnx



### Creamos las 4 tablas de nuestra base de datos.

### Empezamos con peliculas_api...

In [8]:

cnx = establecer_conexion('BD_cinemextract')

query_tabla_api = """CREATE TABLE peliculas_api (

    indice INT,	   
    tipo VARCHAR(20), 
    titulo VARCHAR(250), 
    anyo INT, 
    genero VARCHAR(50),	 
    id_api INT PRIMARY KEY,	
    id_owner VARCHAR(50),
    FOREIGN KEY (id_owner) REFERENCES peliculas_imdb(id_owner))""" 

mycursor = cnx.cursor()


try:
    mycursor.execute(query_tabla_api)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

                                    

Conexión establecida con BD_cinemextract
CMySQLCursor: CREATE TABLE peliculas_api (

    indice..


### Seguimos con la tabla peliculas_imdb

In [27]:
query_tabla_imdb = """CREATE TABLE peliculas_imdb (

    titulo VARCHAR(250),
    puntuacion_imdb FLOAT DEFAULT NULL,
    direccion VARCHAR(200) DEFAULT NULL,
    guionistas VARCHAR(250) DEFAULT NULL,
    argumento TEXT DEFAULT NULL,
    id_owner VARCHAR(50) PRIMARY KEY)""" 

mycursor = cnx.cursor()

try:
    mycursor.execute(query_tabla_imdb)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

CMySQLCursor: CREATE TABLE peliculas_imdb (

    titul..


### Continuamos con la tabla actores


In [9]:

query_tabla_api = """CREATE TABLE actores (

    id_actor INT NOT NULL AUTO_INCREMENT PRIMARY KEY,	
    nombre VARCHAR(200), 
    anyo INT,
    roles VARCHAR (200), 
    peliculas TEXT,
    premios TEXT)""" 

mycursor = cnx.cursor()

try:
    mycursor.execute(query_tabla_api)
    print(mycursor)
    cnx.commit()
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1050 (42S01): Table 'actores' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'actores' already exists


### Y creamos la tabla de los premios oscar.


In [10]:
cnx = establecer_conexion('BD_cinemextract')

query_tabla_oscar = """CREATE TABLE premios_oscar (

    anyo_ceremonia INT PRIMARY KEY,	
    mejor_pelicula TEXT, 
    mejor_director VARCHAR(250),
    mejor_actor VARCHAR(250),
    mejor_actriz VARCHAR(250))""" 


mycursor = cnx.cursor()


try:
    mycursor.execute(query_tabla_oscar)
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

Conexión establecida con BD_cinemextract
1050 (42S01): Table 'premios_oscar' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'premios_oscar' already exists


### Ahora toca insertar los datos en las tablas. Empezamos por peliculas_api

In [11]:
# Establecemos conexión con la base de datos.
cnx = establecer_conexion('BD_cinemextract')

# Creamos un cursor para ejecutar los queries
mycursor = cnx.cursor()

# Creamos un dataframe a partir del archivo CSV con todos los datos extraídos de la API
df = pd.read_csv('peliculas_api_desde_1995_a_2009.csv')

# Creamos la query (sql) y extraemos los valores del .csv

sql = "INSERT INTO peliculas_api(indice, tipo, titulo, anyo, genero, id_api, id_owner)  VALUES (%s, %s, %s, %s, %s, %s, %s)"
val = df.values.tolist()   # Convierto los datos del DataFrame en una lista de listas, donde cada value equivale a la informacion de una fila de la tabla

try:
  # Insertar los datos en la base de datos
  mycursor.executemany(sql, val)
  cnx.commit() 
  print(mycursor.rowcount, "registro/s insertado/s")

except mysql.connector.Error as err:
  print(err)
  print("Error Code:", err.errno)
  print("SQLSTATE", err.sqlstate)
  print("Message", err.msg)

mycursor.close()    # Cerrar el cursosr
cnx.close()         # Cerrar la conexión

Conexión establecida con BD_cinemextract
1062 (23000): Duplicate entry '54935' for key 'peliculas_api.PRIMARY'
Error Code: 1062
SQLSTATE 23000
Message Duplicate entry '54935' for key 'peliculas_api.PRIMARY'


### Llenamos la tabla premios_oscar

In [12]:
# Establecemos conexión con la base de datos.
cnx = establecer_conexion('BD_cinemextract')
    
# Creamos un cursor para ejecutar los queries
mycursor = cnx.cursor()

# Llenamos las tablas a partir del archivo .csv

df = pd.read_csv('tabla_oscar_archivo.csv')

# Creamos las variables sql y val, que serán los parámetros que recibirá nuestro cursor para insertar múltiples registros

sql = "INSERT INTO premios_oscar (anyo_ceremonia, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz) VALUES (%s, %s, %s, %s, %s)"
val = df.values.tolist()   # Convierto el DataFrame a lista de listas, porque aqui no se pueden indicar a "cholon" todos los datos uno a uno.
                          # el .values devuelve las tuplas y .tolist lo pasa a una lista, por lo que ya tengo una lista de tuplas

try:
  # Insertar los datos en la base de datos
  mycursor.executemany(sql, val)
  cnx.commit() # Guardar los cambios y cerrar la conexión
  print(mycursor.rowcount, "registro/s insertado/s")

except mysql.connector.Error as err:
  print(err)
  print("Error Code:", err.errno)
  print("SQLSTATE", err.sqlstate)
  print("Message", err.msg)

Conexión establecida con BD_cinemextract
1062 (23000): Duplicate entry '2000' for key 'premios_oscar.PRIMARY'
Error Code: 1062
SQLSTATE 23000
Message Duplicate entry '2000' for key 'premios_oscar.PRIMARY'


### Llenamos la tabla de los actores.

In [13]:
# Establecemos conexión con la base de datos.
cnx = establecer_conexion('BD_cinemextract')

# Crear un cursor para ejecutar los queries
mycursor = cnx.cursor()

df = pd.read_csv("data_actores 1995_2004.csv")     # Creo un dataframe a partir del .csv

# Creamos las variables sql y val, que serán los parámetros que recibirá nuestro cursor para insertar múltiples registros

sql_actores = "INSERT INTO actores (nombre, anyo, roles, peliculas, premios) VALUES (%s, %s, %s, %s, %s)"
val = df.values.tolist()   # Convierto el DataFrame a lista de listas. El .values devuelve las tuplas y .tolist lo pasa a una lista.

try:
  # Insertar los datos en la base de datos
  mycursor.executemany(sql_actores, val)
  cnx.commit() # Guardar los cambios 
  print(mycursor.rowcount, "registro/s insertado/s")

except mysql.connector.Error as err:
  print(err)
  print("Error Code:", err.errno)
  print("SQLSTATE", err.sqlstate)
  print("Message", err.msg)
  

Conexión establecida con BD_cinemextract
3952 registro/s insertado/s


### Llenamos la tabla de peliculas_imdb

In [37]:
# Establecemos conexión con la base de datos.
cnx = establecer_conexion('BD_cinemextract')

# Crear un cursor para ejecutar los queries
mycursor = cnx.cursor()

# Creamos un dataframe a partir de la lectura del archivo CSV con los datos extraídos de IMDB

df = pd.read_csv('data_peliculas_1995_2009.csv')
 
# Limpiamos los datos y gestionamos los campos vacíos. Convertir la columna Puntuación IMDb de string a float

df['Puntuación IMDb'] = df['Puntuación IMDb'].str.replace(',', '.')
df['Puntuación IMDb'] = df['Puntuación IMDb'].str.replace('No disponible', '-1')
df['Puntuación IMDb'] = df['Puntuación IMDb'].astype(float)

# Gestionamos los campos vacíos en el fichero (en argumento)...

df['Argumento'] = df['Argumento'].astype(str)
df.fillna(value="No disponemos de argumento", inplace=True)  

# Creamos las variables sql y val, que serán los parámetros que recibirá nuestro cursor para insertar múltiples registros

sql = "INSERT INTO peliculas_imdb(titulo, puntuacion_imdb, direccion, guionistas, argumento, id_owner) VALUES (%s, %s, %s, %s, %s, %s)"
val = df.values.tolist()   # Convierto el DataFrame a lista de listas, porque aqui no se pueden indicar a "cholon" todos los datos uno a uno.
                          # el .values devuelve las tuplas y .tolist lo pasa a una lista, por lo que ya tengo una lista de tuplas
try:
  # Insertar los datos en la base de datos
  mycursor.executemany(sql, val)
  cnx.commit() # Guardar los cambios y cerrar la conexión
  print(mycursor.rowcount, "registro/s insertado/s")

except mysql.connector.Error as err:
  print(err)
  print("Error Code:", err.errno)
  print("SQLSTATE", err.sqlstate)
  print("Message", err.msg)

mycursor.close()    # Cerrar el cursosr
cnx.close()         # Cerrar la conexión


Conexión establecida con BD_cinemextract
652 registro/s insertado/s


In [15]:

# Y finalmente cerramos la conexión.

cnx.close() 